In [1]:
%matplotlib inline

In [2]:
# Choose whether to plot cases, deaths, or both (as column titles)

columns = ["diagnosed", "killed"]

In [3]:
# Specify which model to use: "exp" for exponential, "log" for logistic

models = {"diagnosed": "log",
          "killed": "log"}

In [4]:
equations = {"exp": "$f(t) = a (1 + b)^t$\n$a = {0:.4f} \pm {2:.5f}$\n$b = {1:.4f} \pm {3:.6f}$", 
             "log": "$f(t) = c / (1 + \exp((b - t)/a))$\n$a = {0:.3f} \pm {3:.4f}$\n$b = {1:.2f} \mp {4:.4f}$\n$c = {2:.0f}. \pm {5:.2f}$"}

In [5]:
# Set colors for the plot

colors = {"diagnosed": "red",
          "killed": "black"}

In [6]:
# Specify names for dataset (input) and plot (output)

dataname = "us_md_montgomery.csv"
imgname = "us_md_montgomery.png"

In [7]:
# Prepare dicts for stats

residuals = {"diagnosed": [],
             "killed": []}

chi_sq_red = {"diagnosed": 1.0,
              "killed": 1.0}

In [8]:
# Everything else is details

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import strptime
from datetime import date
from string import Template
from scipy.optimize import curve_fit
from scipy.stats import describe, chisquare, t
from ipywidgets import IntSlider, interact
from matplotlib import style
style.use("seaborn")

In [9]:
# Define the model equation and its Jacobian

def f_exp(t, a, b):
    # Exponential growth law, $f(t) = a * (1 + b) ^ t$,
    # where $a$ is the number of cases at $t=0$ and $b$ is the growth rate.
    return a * (1 + b) ** t

def df_exp(t, a, b):
    # Jacobian: df/dp for p=(a, b)
    return np.array([(1 + b) ** t,
 (a * t * (1 + b) ** t) / (1 + b)]).T

def f_log(t, a, b, c):
    # Logistic growth law, $f(t) = c / (exp((b - t)/a) + 1)$
    return c / (np.exp((b - t)/a) + 1)

def df_log(t, a, b, c):
    # Jacobian: df/dp for p=(a, b, c)
    return np.array([c*(b - t)*np.exp((b - t)/a)/(a**2*(np.exp((b - t)/a) + 1)**2),
 -c*np.exp((b - t)/a)/(a*(np.exp((b - t)/a) + 1)**2),
 1/(np.exp((b - t)/a) + 1)]).T

In [10]:
def sigfig(x, n):
    # Round a float, x, to n significant figures.
    # Source: https://github.com/corriander/python-sigfig
    n = int(n)

    e = np.floor(np.log10(np.abs(x)) - n + 1) # exponent, 10 ** e
    shifted_dp = x / (10 ** e) # decimal place shifted n d.p.
    return np.around(shifted_dp) * (10 ** e) # round and revert

In [11]:
# Data

data = pd.read_csv(dataname)

ndays = len(data)

In [12]:
def fit_dor(dor):
    fig = plt.figure(figsize=(10, 7))

    plt.suptitle("COVID-19 in Montgomery County, Maryland, USA", fontweight="bold")
    plt.title("github.com/tkphd/covid19-curve-your-county", style="oblique")
    plt.xlabel("Day of Record")
    plt.ylabel("Number of People")
    
    start = strptime(data["date"].iloc[0], "%Y-%m-%d")
    start = date(start.tm_year, start.tm_mon, start.tm_mday).toordinal()
    today = strptime(data["date"].iloc[dor], "%Y-%m-%d")
    today = date(today.tm_year, today.tm_mon, today.tm_mday).toordinal()
    
    key = "diagnosed"
    model = models[key]
    f  = f_exp
    df = df_exp

    if model == "log":
        f  = f_log
        df = df_log

    allPeople = np.array(data[key])
    allDays   = np.zeros_like(allPeople)
    for i in range(len(allPeople)):
        day = strptime(data["date"].iloc[i], "%Y-%m-%d")
        allDays[i] = date(day.tm_year, day.tm_mon, day.tm_mday).toordinal() - start

    plt.scatter(allDays, allPeople, marker=".", s=10, color="white", edgecolors=colors[key], zorder=10)
    plt.xlim([0,   1 + np.amax(allDays)])
    plt.ylim([0, 500 + np.amax(allPeople)])
    # Subset data from slider

    t = allDays[:dor]
    y = allPeople[:dor]

    # Levenburg-Marquardt Least-Squares Fit
    """
    Note that sigma represents the relative error associated with each data point. By default, `curve_fit`
    will assume an array of ones (constant values imply no difference in error), which is probably
    incorrect: given sparsity of testing, there's considerable uncertainty, and the earlier numbers may
    be lower than the truth to a greater extent than the later numbers. Quantifying this error by some
    non-trivial means for each datapoint would produce much more realistic uncertainty bands in the
    final plots.
    """
    p, pcov = curve_fit(f, t, y, sigma=None, method="lm", jac=df, maxfev=1000)
    coef = describe(pcov)
    perr = np.sqrt(np.diag(pcov))

    # Reduced chi-square goodness of fit
    ## https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic

    chisq, chip = chisquare(y, f(t, *p))
    ndof = len(y) - len(p) - 1

    residuals[key] = f(t, *p) - y
    chi_sq_red[key] = chisq / ndof

    # Confidence Band: dfdp represents the partial derivatives of the model with respect to each parameter p (i.e., a and b)

    t_hat = np.linspace(0, t[-1] + 7, 100)
    y_hat = f(t_hat, *p)

    if (models[key] == "log"):
        perr[1] *= -1

    upr_p = p + perr
    lwr_p = p - perr

    if (models[key] == "log"):
        perr[1] *= -1

    upper = f(t_hat, *upr_p)
    lower = f(t_hat, *lwr_p)

    it = np.argsort(t_hat)
    plt.plot(t_hat[it], y_hat[it], c=colors[key], lw=1, zorder=5, label=key.capitalize())
    plt.fill_between(
        t_hat[it], upper[it], y_hat[it], edgecolor=None, facecolor="silver", zorder=1
    )
    plt.fill_between(
        t_hat[it], lower[it], y_hat[it], edgecolor=None, facecolor="silver", zorder=1
    )

    # Predictions

    dx = 0.25
    dt = 14

    tomorrow = date.fromordinal(today + 1)
    nextWeek = date.fromordinal(today + 7)

    t_hat = np.array([tomorrow.toordinal() - start, nextWeek.toordinal() - start])
    y_hat = f(t_hat, *p)

    upper = f(t_hat, *upr_p)
    lower = f(t_hat, *lwr_p)

    s = sigfig(p, 4)
    serr = sigfig(perr, 4)

    # Overlay projections on plot
    plt.text(
        t_hat[0] - dt,
        y_hat[0],
        "{0}/{1}: ({2:.0f} < {3:.0f} < {4:.0f})".format(
        tomorrow.month, tomorrow.day, lower[0], y_hat[0], upper[0]
        ),
        color=colors[key],
        va="center",
        ha="center",
        zorder=4,
        bbox=dict(boxstyle="round", ec="gray", fc="ghostwhite", linewidth=dx)
    )

    hw = 12
    hl = t_hat[1] / 100

    plt.arrow(
        t_hat[0] - dt,
        y_hat[0],
        dt - dx + 0.0625,
        0,
        fc="black",
        ec="black",
        head_width=hw,
        head_length=hl,
        overhang=dx,
        length_includes_head=True, 
        linewidth=0.5,
        zorder=3,
    )

    # Save figure    

    plt.legend(loc="center left")
    plt.show()

interact(fit_dor, dor = IntSlider(
    value = ndays - 1,
    min = 20,
    max = ndays - 1,
    step = 1,
    description = 'Day of Record:',
    disabled = False,
    continuous_update = False,
    orientation = 'horizontal',
    readout = True,
    readout_format = 'd')
)    

interactive(children=(IntSlider(value=62, continuous_update=False, description='Day of Record:', max=62, min=2…

<function __main__.fit_dor(dor)>